# How to inspect runnables

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)

:::

Once you create a runnable with [LangChain Expression Language](/docs/concepts/#langchain-expression-language), you may often want to inspect it to get a better sense for what is going on. This notebook covers some methods for doing so.

This guide shows some ways you can programmatically introspect the internal steps of chains. If you are instead interested in debugging issues in your chain, see [this section](/docs/how_to/debugging) instead.

First, let's create an example chain. We will create one that does retrieval:

In [1]:
%pip install -qU langchain langchain-openai faiss-cpu tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## Get a graph

You can use the `get_graph()` method to get a graph representation of the runnable:

In [3]:
chain.get_graph()

Graph(nodes={'784a4e6f9fb248b69b37ec45bc9af211': Node(id='784a4e6f9fb248b69b37ec45bc9af211', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.utils.RunnableParallel<context,question>Input'>, metadata=None), '249731e0668e4ac09c6bad0260793977': Node(id='249731e0668e4ac09c6bad0260793977', name='Parallel<context,question>Output', data=<class 'langchain_core.runnables.utils.RunnableParallel<context,question>Output'>, metadata=None), 'b54a8b269fec40d08dd72326b33e66ce': Node(id='b54a8b269fec40d08dd72326b33e66ce', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117cbb510>, search_kwargs={}), metadata=None), '9c73396539154880930d166d52941aa9': Node(id='9c73396539154880930d166d52941aa9', name='Passthrough', data=RunnablePassthrough(), metadata=None), '2edf6faa3b374ac8973c14d776f0ca0d': Node(id='2edf6faa3b374ac8973c14d776f0ca0d', name='ChatPromptTemplate', 

## Print a graph

While that is not super legible, you can use the `print_ascii()` method to show that graph in a way that's easier to understand:

In [4]:
chain.get_graph().print_ascii()

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

## Get the prompts

You may want to see just the prompts that are used in a chain with the `get_prompts()` method:

In [5]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])]

## Next steps

You've now learned how to introspect your composed LCEL chains.

Next, check out the other how-to guides on runnables in this section, or the related how-to guide on [debugging your chains](/docs/how_to/debugging).